In [90]:
import pandas as pd
import sklearn as sk
import numpy as np

In [91]:
train_data = pd.read_csv("tit/train.csv")
test_data = pd.read_csv("tit/test.csv")
all_data = [train_data, test_data]

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [92]:
train_data = train_data.drop(['Ticket', 'Cabin', 'Fare', 'Embarked'], axis=1)
test_data = test_data.drop(['Ticket', 'Cabin', 'Fare', 'Embarked'], axis=1)
all_data = [train_data, test_data]

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0


In [93]:
for dataset in all_data:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
for dataset in all_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5}
for dataset in all_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,1


In [94]:
train_data = train_data.drop(['Name', 'PassengerId'], axis=1)
test_data = test_data.drop(['Name'], axis=1)
all_data = [train_data, test_data]

train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Title
0,0,3,male,22.0,1,0,1
1,1,1,female,38.0,1,0,3
2,1,3,female,26.0,0,0,2
3,1,1,female,35.0,1,0,3
4,0,3,male,35.0,0,0,1


In [95]:
for dataset in all_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Title
0,0,3,1,22.0,1,0,1
1,1,1,0,38.0,1,0,3
2,1,3,0,26.0,0,0,2
3,1,1,0,35.0,1,0,3
4,0,3,1,35.0,0,0,1


In [96]:
pred_age = np.zeros((2,3))
pred_age

for dataset in all_data:
    for i in range(0, 2):
        for j in range(0, 3):
            pred_age[i,j] = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna().median()
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = pred_age[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Title
0,0,3,1,22,1,0,1
1,1,1,0,38,1,0,3
2,1,3,0,26,0,0,2
3,1,1,0,35,1,0,3
4,0,3,1,35,0,0,1


In [97]:
X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()

In [98]:
logistic_regression = sk.linear_model.LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, Y_train)
print(logistic_regression.score(X_train, Y_train))

0.8170594837261503


In [99]:
svc = sk.svm.SVC(gamma='auto').fit(X_train, Y_train)
print(svc.score(X_train, Y_train))

0.8597081930415263


In [100]:
knn = sk.neighbors.KNeighborsClassifier(n_neighbors = 2).fit(X_train, Y_train)
print(knn.score(X_train, Y_train))

0.8630751964085297


In [101]:
gaussian = sk.naive_bayes.GaussianNB().fit(X_train, Y_train)
print(gaussian.score(X_train, Y_train))

0.8114478114478114


In [102]:
random_forest = sk.ensemble.RandomForestClassifier(n_estimators=200).fit(X_train, Y_train)
print(random_forest.score(X_train, Y_train))

0.9225589225589226


In [103]:
predictions = random_forest.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": predictions
})

submission.to_csv("titanic-submission.csv", index=False)